- We loose structure of dialogue using the tokenizer in preprocess_data

- Emotions encoded using MultiLabelBinarizer doesn't tell us
  anymore how many times a single emotion is present in the dialogue
  and where it is

In [64]:
"""
!pip install torch==1.13.0+cu116 --extra-index-url https://download.pytorch.org/whl/cu116
!pip install transformers==4.30.0
!pip install datasets==2.13.2
!pip install accelerate -U
!pip install evaluate
"""

'\n!pip install torch==1.13.0+cu116 --extra-index-url https://download.pytorch.org/whl/cu116\n!pip install transformers==4.30.0\n!pip install datasets==2.13.2\n!pip install accelerate -U\n!pip install evaluate\n'

In [65]:
# system packages
from pathlib import Path
import shutil
import urllib
import tarfile
import sys
import os
# data and numerical management packages
import pandas as pd
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
# useful during debugging (progress bars)
from tqdm import tqdm
from transformers import set_seed

seed = 852

random.seed(seed)
np.random.seed(seed)
set_seed(seed)

In [66]:
dataset_folder = Path.cwd().joinpath("Datasets")
dataset_path = dataset_folder.joinpath('MELD_train_efr.json')

df = pd.read_json(dataset_path)

In [67]:
df

,episode,speakers,emotions,utterances,triggers
0,utterance_0,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise]",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 1.0, 0.0]"
1,utterance_1,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]"
2,utterance_2,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
3,utterance_3,"[Chandler, The Interviewer, Chandler, The Inte...","[neutral, neutral, neutral, neutral, surprise,...",[also I was the point person on my company's t...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,utterance_4,"[Joey, Rachel, Joey, Rachel]","[surprise, sadness, surprise, fear]",[But then who? The waitress I went out with la...,"[0.0, 0.0, 1.0, 0.0]"
...,...,...,...,...,...
3995,utterance_3995,"[Chandler, All, Monica, Chandler, Ross, Chandl...","[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3996,utterance_3996,"[Chandler, All, Monica, Chandler, Ross, Chandl...","[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3997,utterance_3997,"[Chandler, All, Monica, Chandler, Ross, Chandl...","[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3998,utterance_3998,"[Chandler, All, Monica, Chandler, Ross, Chandl...","[neutral, joy, neutral, neutral, surprise, dis...","[Hey., Hey!, So how was Joan?, I broke up with...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [68]:
# Replace None with 0.0 trigger

triggers = df['triggers']
for row in range(triggers.shape[0]):
    for trigger in range(len(triggers[row])):
        if triggers[row][trigger] == None:
            triggers[row][trigger] = 0.0

df['triggers'] = triggers

In [69]:
# Unique values for 'emotions'

emotions = df['emotions'].explode().unique()
emotions = emotions.tolist()
emotions

['neutral', 'surprise', 'fear', 'sadness', 'joy', 'disgust', 'anger']

In [70]:
# Unique values for 'triggers'

triggers = df['triggers'].explode().unique()
triggers

array([0.0, 1.0], dtype=object)

In [71]:
# Itterating through dialogues to find the one (index) with the larger number of utterances (max_len_dialogue)

var = df['utterances']

max_len_dialogue = 0
index = 0
for idx, dialogue in enumerate(var):
  if len(dialogue) > max_len_dialogue:
    max_len_dialogue = len(dialogue)
    index = idx
max_len_dialogue,index

(24, 219)

In [72]:
# Itterating through utterances over all dialogues to find the utterance (idx_sentence) with the largest length (utterances)

max_len_sentence = 0
index = 0
for idx, dialogue in enumerate(var):
  for idx_sentence, utterance in enumerate(dialogue):
    if len(utterance.split()) > max_len_sentence:
      max_len_sentence = len(utterance.split())
      index = idx,idx_sentence
max_len_sentence,index

(69, (1675, 11))

In [73]:
# Drop the 'speakers' column

df = df.drop(columns=['speakers'])

In [74]:
# Padding of dialogues

padded_dialogues = [seq + ['[PAD]'] * (max_len_dialogue - len(seq)) for seq in var]

In [75]:
# Replace old 'utterances' column with padded_dialogues

df['utterances'] = padded_dialogues

In [76]:
from sklearn.preprocessing import LabelBinarizer

label_binarizer = LabelBinarizer()
label_binarizer.fit(emotions)

binarizer_test = []
for emotions in df['emotions']:
    binarized_emotions = label_binarizer.transform(emotions)
    binarized_emotions = binarized_emotions.tolist()[0]
    binarizer_test.append(binarized_emotions)

In [77]:
df['emotions'] = binarizer_test

# Check that it has received the right classes

label_binarizer.classes_

array(['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness',
       'surprise'], dtype='<U8')

In [78]:
# Padding of emotions

padded_emotions = [seq + [0] * (max_len_dialogue - len(seq)) for seq in df['emotions']]

df['emotions'] = padded_emotions

In [79]:
# Padding of triggers

padded_triggers = [seq + [0] * (max_len_dialogue - len(seq)) for seq in df['triggers']]

df['triggers'] = padded_triggers

In [80]:
import torch
from torch.utils.data import Dataset
import torch.nn.functional as F


class CustomDataset(Dataset):
    # max_len means that input shouldn't be of length greater than that number
    def __init__(self, utterances, emotions, triggers, tokenizer, max_len=max_len_dialogue):
        self.utterances = utterances
        self.emotions = emotions
        self.triggers = triggers
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.utterances)

    def __getitem__(self, idx):
        utterance = self.utterances[idx]
        emotion = self.emotions[idx]
        trigger = self.triggers[idx]

        # Tokenize and pad the dialogue
        utterance_inputs = self.tokenizer(utterance, max_length=self.max_len, padding='max_length', truncation=True, return_tensors='pt')

        # Tokenize and pad the emotion labels
        emotion_inputs = self.tokenizer(emotion, max_length=self.max_len, padding='max_length', truncation=True, return_tensors='pt')

        # Tokenize and pad the trigger labels
        trigger_inputs = self.tokenizer(trigger, max_length=self.max_len, padding='max_length', truncation=True, return_tensors='pt')

        return {
            'utterance_input_ids': utterance_inputs['input_ids'].squeeze(),
            'utterance_attention_mask': utterance_inputs['attention_mask'].squeeze(),
            'emotion_input_ids': emotion_inputs['input_ids'].squeeze(),
            'emotion_attention_mask': emotion_inputs['attention_mask'].squeeze(),
            'trigger_input_ids': trigger_inputs['input_ids'].squeeze(),
            'trigger_attention_mask': trigger_inputs['attention_mask'].squeeze(),
        }

In [81]:
from sklearn.model_selection import train_test_split

train_data, temp_data = train_test_split(df, train_size=0.8, shuffle=False)
val_data, test_data = train_test_split(temp_data, test_size=0.5, shuffle=False)

In [82]:
from torch import nn
from transformers import BertModel, BertConfig

class CustomBERTModel(torch.nn.Module):
    def __init__(self, freeze_embedding=True, num_emotion_labels=2, num_trigger_labels=2):
        super(CustomBERTModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        
        if freeze_embedding:
            for param in self.bert.embeddings.parameters():
                param.requires_grad = False

        self.emotion_classifier = torch.nn.Linear(self.bert.config.hidden_size, num_emotion_labels)
        self.trigger_classifier = torch.nn.Linear(self.bert.config.hidden_size, num_trigger_labels)

    def forward(self, utterance_input_ids, utterance_attention_mask,
                emotion_input_ids, emotion_attention_mask,
                trigger_input_ids, trigger_attention_mask):
        utterance_outputs = self.bert(input_ids=utterance_input_ids, attention_mask=utterance_attention_mask)
        emotion_outputs = self.bert(input_ids=emotion_input_ids, attention_mask=emotion_attention_mask)
        trigger_outputs = self.bert(input_ids=trigger_input_ids, attention_mask=trigger_attention_mask)
        
        pooled_utterance_output = utterance_outputs.pooler_output
        pooled_emotion_output = emotion_outputs.pooler_output
        pooled_trigger_output = trigger_outputs.pooler_output

        emotion_logits = self.emotion_classifier(pooled_emotion_output)
        trigger_logits = self.trigger_classifier(pooled_trigger_output)

        return emotion_logits, trigger_logits

In [83]:
from transformers import AutoTokenizer
from torch.utils.data import DataLoader

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

train_utterances = train_data['utterances']
train_emotions = train_data['emotions']
train_triggers = train_data['triggers']

val_utterances = val_data['utterances']
val_emotions = val_data['emotions']
val_triggers = val_data['triggers']

train_data = CustomDataset(train_utterances, train_emotions, train_triggers, tokenizer)
val_data = CustomDataset(val_utterances, val_emotions, val_triggers, tokenizer)

train_dataloader = DataLoader(train_data, batch_size=24, shuffle=False)
val_dataloader = DataLoader(val_data, batch_size=24, shuffle=False)

In [84]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_collator

DataCollatorWithPadding(tokenizer=BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

In [85]:
from sklearn.metrics import classification_report
from torch.nn import BCEWithLogitsLoss

num_emotion_labels = 2
num_trigger_labels = 2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CustomBERTModel(num_emotion_labels, num_trigger_labels)
model.to(device)

# Use BCEWithLogitsLoss for multi-label classification
criterion = BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

num_epochs = 2

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        utterance_input_ids = batch['utterance_input_ids'].to(device)
        utterance_attention_mask = batch['utterance_attention_mask'].to(device)
        emotion_labels = batch['emotion_input_ids'].float().to(device)  # Ensure labels are float for BCEWithLogitsLoss
        trigger_labels = batch['trigger_input_ids'].float().to(device)  # Ensure labels are float for BCEWithLogitsLoss

        optimizer.zero_grad()

        emotion_logits, trigger_logits = model(utterance_input_ids, utterance_attention_mask,
                                              utterance_input_ids, utterance_attention_mask,  # Repeating for emotions
                                              utterance_input_ids, utterance_attention_mask)  # Repeating for triggers

        # Use BCEWithLogitsLoss for multi-label classification
        emotion_loss = criterion(emotion_logits, emotion_labels)
        trigger_loss = criterion(trigger_logits, trigger_labels)

        total_loss = emotion_loss + trigger_loss
        total_loss.backward()
        optimizer.step()

    model.eval()
    all_emotion_preds, all_trigger_preds = [], []
    all_emotion_labels, all_trigger_labels = [], []

    with torch.no_grad():
        for batch in val_dataloader:
            utterance_input_ids = batch['utterance_input_ids'].to(device)
            utterance_attention_mask = batch['utterance_attention_mask'].to(device)
            emotion_labels = batch['emotion_input_ids'].float().to(device)  # Ensure labels are float for BCEWithLogitsLoss
            trigger_labels = batch['trigger_input_ids'].float().to(device)  # Ensure labels are float for BCEWithLogitsLoss

            optimizer.zero_grad()

            emotion_logits, trigger_logits = model(utterance_input_ids, utterance_attention_mask,
                                                utterance_input_ids, utterance_attention_mask,  # Repeating for emotions
                                                utterance_input_ids, utterance_attention_mask)  # Repeating for triggers

            # Use BCEWithLogitsLoss for multi-label classification
            emotion_loss = criterion(emotion_logits, emotion_labels)
            trigger_loss = criterion(trigger_logits, trigger_labels)

            total_loss = emotion_loss + trigger_loss

            # Accumulate predictions and labels
            all_emotion_preds.append(torch.sigmoid(emotion_logits).cpu().numpy())
            all_trigger_preds.append(torch.sigmoid(trigger_logits).cpu().numpy())
            all_emotion_labels.append(emotion_labels.cpu().numpy())
            all_trigger_labels.append(trigger_labels.cpu().numpy())

    print(f'Epoch {epoch + 1}/{num_epochs}')
    print('Emotion Classification Report:')
    print(classification_report(np.concatenate(all_emotion_labels, axis=0), np.concatenate(all_emotion_preds, axis=0) > 0.5))
    print('Trigger Classification Report:')
    print(classification_report(np.concatenate(all_trigger_labels, axis=0), np.concatenate(all_trigger_preds, axis=0) > 0.5))


ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [ ]:
from sklearn.metrics import classification_report
from torch.nn import BCEWithLogitsLoss

num_emotion_labels = 2
num_trigger_labels = 2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CustomBERTModel(model, num_emotion_labels, num_trigger_labels)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

num_epochs = 2

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        emotion_labels = batch['emotion'].to(device)
        trigger_labels = batch['trigger'].to(device)

        optimizer.zero_grad()

        emotion_logits, trigger_logits = model(input_ids, attention_mask)

        # Use CrossEntropyLoss for single-label classification
        emotion_loss = criterion(emotion_logits, emotion_labels)
        trigger_loss = criterion(trigger_logits, trigger_labels)

        total_loss = emotion_loss + trigger_loss
        total_loss.backward()
        optimizer.step()

    model.eval()
    all_emotion_preds, all_trigger_preds = [], []
    all_emotion_labels, all_trigger_labels = [], []

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            emotion_labels = batch['emotion'].to(device)
            trigger_labels = batch['trigger'].to(device)

            emotion_logits, trigger_logits = model(input_ids, attention_mask)

            # Use CrossEntropyLoss for single-label classification
            emotion_loss = criterion(emotion_logits, emotion_labels)
            trigger_loss = criterion(trigger_logits, trigger_labels)

            total_loss = emotion_loss + trigger_loss

            # Accumulate predictions and labels
            all_emotion_preds.append(emotion_logits.argmax(dim=1).cpu().numpy())
            all_trigger_preds.append(trigger_logits.argmax(dim=1).cpu().numpy())
            all_emotion_labels.append(emotion_labels.cpu().numpy())
            all_trigger_labels.append(trigger_labels.cpu().numpy())

    print(f'Epoch {epoch + 1}/{num_epochs}')
    print('Emotion Classification Report:')
    print(classification_report(all_emotion_labels, all_emotion_preds))
    print('Trigger Classification Report:')
    print(classification_report(all_trigger_labels, all_trigger_preds))

RuntimeError: stack expects each tensor to be equal size, but got [24, 24] at entry 0 and [24, 21] at entry 4